In [17]:

import pandas as pd
import joblib
from tensorflow import keras


MODEL_PATH    = 'best_multilabel_keras_model.keras'
SCALER_PATH   = 'scaler.joblib'
MLB_PATH      = 'mlb_binarizer.joblib'
FEATURES_PATH = 'feature_columns.joblib'


def preprocess_new_data(df_raw, feature_columns, scaler):


    df = df_raw.rename(columns={
        'Ages': 'Usia',
        'Gender': 'Jenis_Kelamin',
        'Height': 'Tinggi_cm',
        'Weight': 'Berat_kg',
        'Protein': 'Protein_g',
        'Sugar': 'Gula_g',
        'Sodium': 'Sodium_mg',
        'Calories': 'Kalori',
        'Carbohydrates': 'Karbohidrat_g',
        'Fiber': 'Serat_g',
        'Fat': 'Lemak_g'
    })


    numeric_cols = ['Usia','Tinggi_cm','Berat_kg','Protein_g','Gula_g',
                    'Sodium_mg','Kalori','Karbohidrat_g','Serat_g','Lemak_g']
    categorical_cols = ['Jenis_Kelamin']


    missing = [c for c in numeric_cols + categorical_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Kolom input hilang: {missing}")


    df[numeric_cols] = scaler.transform(df[numeric_cols])

    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

    df = df.reindex(columns=feature_columns, fill_value=0)
    return df


if __name__ == '__main__':

    model = keras.models.load_model(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    mlb = joblib.load(MLB_PATH)
    feature_columns = joblib.load(FEATURES_PATH)


    sample = {
        'Ages': [50],
        'Gender': ['Female'],
        'Height': [165],
        'Weight': [63],
        'Protein': [50],
        'Sugar': [50],
        'Sodium': [25],
        'Calories': [2500],
        'Carbohydrates': [300],
        'Fiber': [22],
        'Fat': [70]
    }
    df_new = pd.DataFrame(sample)


    X_new = preprocess_new_data(df_new, feature_columns, scaler)
    proba = model.predict(X_new)
    y_pred = (proba > 0.5).astype(int)
    labels = mlb.inverse_transform(y_pred)


    print("Probabilitas per label:", proba)
    print("Prediksi biner (0/1):", y_pred)
    print("Label terprediksi:", labels[0] or ['Normal'])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Probabilitas per label: [[5.5377976e-16 7.0805453e-16 5.6312793e-01 5.1587737e-01 7.6924849e-01
  2.2707691e-10 9.9521953e-01 5.9704487e-13]]
Prediksi biner (0/1): [[0 0 1 1 1 0 1 0]]
Label terprediksi: ('Heart Disease', 'Hypertension', 'Kidney Disease', 'Weight Gain')
